In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
from sklearn.model_selection import train_test_split
print(cp.__version__)

In [ ]:
np.random.seed(20)

**Parsing Data**

In [ ]:
# filename = "mat_comp_small"
filename = "mat_comp"
n, m, k = 0, 0, 0

f = open(filename, "r")
n, m, k = map(int, f.readline().strip().split(" "))
f.close()

ratings = pd.read_csv(filename, delimiter=" ",skiprows=1, nrows=k, names=["i", "j", "rating"])
q = pd.read_csv(filename, delimiter=" ",skiprows=k+1, nrows=1, names=["1"])["1"][0]
queries = pd.read_csv(filename, delimiter=" ",skiprows=k+2, nrows=q, names=["i", "j"])

In [ ]:
print(queries)
print(ratings)
ratings.describe()

**Train Test Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ratings[["i", "j"]], ratings["rating"], test_size=0.2, random_state=49)
X_test = np.asarray(X_test)
X_train = np.asarray(X_train)
y_train = np.asarray(y_train).reshape(-1, 1)
y_test = np.asarray(y_train).reshape(-1, 1)

print(np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test))
print(X_test[:5])
print(y_test[:5])

**Constructing M**

In [ ]:
M = np.zeros([n, m], dtype=float)
for indices, rating in zip(X_train, y_train):
    M[indices[0]-1][indices[1]-1] = rating
print(f"M = \n{M}\nM_shape = {np.shape(M)}\n\n")
print(n, m)
print(f"Queries = \n{queries}")

**Initializations**

In [ ]:
"""SVD Initializations"""
U, singular_values, V = np.linalg.svd(M, full_matrices=False)
S = np.zeros((np.shape(U)[1], np.shape(V)[0]))
np.fill_diagonal(S, singular_values)
X1, Y1 = U@S, V
# print(f"U shape: {np.shape(U)}")
# print(f"S shape: {np.shape(S)}")
# print(f"V shape: {np.shape(V)}")
# print(f"X1 shape: {np.shape(X1)}")
# print(f"Y1 shape: {np.shape(Y1)}")

r = 15
MAX_ITERS = 30
lr = 0.001
X_init = np.random.uniform(0, 5, (m, r))
M_input = M.transpose()

print(np.shape(X_init))

**Alternating Minimizaiton**

In [ ]:
def alternating_minimization(M, r, max_iter, X_init):
    X, Y = X_init, None
    residual = np.zeros(MAX_ITERS)
    for iter_num in range(1, max_iter + 1):
        # For odd iterations, treat X constant, optimize over Y.
        if iter_num % 2 == 1:
            Y = cp.Variable(shape=(r, n))
            constraint = [Y >= 0, Y <= 3]
        # For even iterations, treat Y constant, optimize over X.
        else:
            X = cp.Variable(shape=(m, r))
            constraint = [X >= 0, X <= 3]
            
        obj = cp.Minimize(cp.norm(M - X@Y, 'fro'))
        prob = cp.Problem(obj, constraint)
        prob.solve(solver=cp.SCS, max_iters=5000)
        
        if prob.status != cp.OPTIMAL:
            raise Exception("Solver did not converge!")
        
        print('Iteration {}, residual norm: {}'.format(iter_num, round(prob.value, 2)))
        residual[iter_num-1] = prob.value

        # Convert variable to NumPy array constant for next iteration.
        if iter_num % 2 == 1:
            Y = Y.value
        else:
            X = X.value
    
    return X, Y

**Gradient Descent**

In [ ]:
def gradient_descent(M, X_init, Y_init, lr = 0.001, iterations=10000):
    """
    Gradient Descent 1: This approach did not work because the in the loss funciton,
    the M we used represented empty results as 0s. This caused gradient descent
    to optimize the indices of the queries to 0. Therefore this approach does not
    work.
    """
    X, Y = X_init, Y_init
    cur_M = X @ Y.T
    for i in range(iterations):
        if i % 1000 == 0: print(f"{round((i / iterations) * 100, 2)}% complete \n {cur_M}")
        X -= lr * (-2 * (M - X@Y.T) @ Y)
        Y -= lr * (-2 * (M.T - Y@X.T) @ X)
        cur_M = X @ Y.T
    return X, Y

In [ ]:
def gradient_descent2(M, X_init, Y_init, nonzeros, lr = 0.001, iterations=10000):
    """
    Gradient Descent 2: This approach seeks to optimize using a more granular view
    to the matrices X and Y. The loss is approximately to minimize M - X@Y
    """
    X, Y = X_init, Y_init
    ii, jj = nonzeros
    ii -= 1
    jj -= 1
    for epoch in range(iterations):
        if epoch % 100 == 0:
            print(f"epoch: {epoch}")
            print(f"cur_m\n: {X@Y}\n")
            print(f"cur_m[0][3]: {(X@Y)[0][3]}\n")
        for i, j in zip(ii,jj):
            e_ij = M[i][j] - np.dot(X[i,:], Y[:,j])
            grad_X = -2 * e_ij * Y[:,j]
            grad_Y = -2 * e_ij * X[i,:]
            X[i,:] -= lr * grad_X
            Y[:,j] -= lr * grad_Y
    return X, Y

**Loss Function**

In [ ]:
def test_loss(X_test, y_test, predict_m, verbose=False):
    """
    predicted_m: nxm ndarray containing recommendation predictions for indices in X_test. y_test contains ground truth ratings
    """
    squared_error = 0
    S = len(X_test)
    for indices, true in zip(X_test, y_test):
        if verbose: print(f"true: {true[0]} predicted: {round(predict_m[indices[0]-1][indices[1]-1], 2)}")
        squared_error += (true[0] - predict_m[indices[0]-1][indices[1]-1]) ** 2

    loss = (1/S) * squared_error
    print(f"loss: {loss}")
    print(f"grade: {min(4, max(0, 4*(1.8 - loss)))}")

**Gradient Descent**

In [ ]:
xinit, yinit = np.random.uniform(0, 0.7, (n, r)), np.random.uniform(0, 0.7, (m, r))
X_descent, Y_descent = gradient_descent(M, xinit, yinit, learning_rate=lr, iterations=80000)
gradient_descent_M1 = X_descent@Y_descent.transpose()

In [ ]:
#initialize X, Y, and nonzero indices
xinit, yinit = np.random.uniform(0, 0.7, (n, r)), np.random.uniform(0, 0.7, (r, m))
nonzero_indices = M.nonzero()

X_descent2, Y_descent2 = gradient_descent2(M, xinit, yinit, nonzero_indices, lr=lr, iterations=5000)
gradient_descent_M2 = X_descent2@Y_descent2

In [ ]:
"""Testing Loss for Gradient Descents"""
test_loss(X_test, y_test, gradient_descent_M2, verbose=True) #gd2

**Alternating Minimization**

In [ ]:
"""Alternating Minimization"""
X_alter, Y_alter = alternating_minimization(M_input, r, MAX_ITERS, X_init)
alternating_minimization_M = (X_alter@Y_alter).transpose() #transposing because in alternaitng minimization, X@Y results in M_transpose.

In [ ]:
"""Testing Loss for Alternating Minimization"""
test_loss(X_test, y_test, alternating_minimization_M)

**SKLEARN NMF**

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=7, init='random', random_state=2, max_iter=10000, beta_loss=2)
W = model.fit_transform(M)
H = model.components_

In [ ]:
sklearn_nmf_M = W@H
print(np.shape(sklearn_nmf_M))

scale = lambda t: t * 5 if (t * 3.5 < 4.5) else t
vscale = np.vectorize(scale)
m = vscale(sklearn_nmf_M)

test_loss(X_test, y_test, sklearn_nmf_M)

**Uniform LOL**

In [ ]:
uniform3 = np.random.uniform(3.3, 3.3, np.shape(M))
test_loss(X_test, y_test, uniform3)

**Final Predictions**

In [ ]:
def make_predictions(m_pred):    
    query_prediction = np.zeros(len(queries.index))
    with open("./mat_comp_ans", "w") as file:
        for index, row in queries.iterrows():
            file.write(str(3.3) + "\n")
            # query_prediction[index] = m_pred[row['i']-1][row['j']-1]
    print(len(queries))

make_predictions(uniform3)